# Generating Objects w/ `Variational Auto Encoder (VAE)`

### Import dependencies

In [1]:
import os
import sys
# import datetime as dt

import numpy as np
# import tensorflow as tf

from dataset import ImageDataset

### Load dataset

In [2]:
data_dir = 'datasets/101_ObjectCategories/'
save_file = 'saved/data.pkl'

In [ ]:
data = ImageDataset(data_dir=data_dir, size=128, grayscale=True, flatten=False)
data.create()
data.save(save_file=save_file, force=True)
# data = data.load(save_file=save_file)

1 of 102 labels	135 of 468 images...

In [ ]:
data.images.shape

In [ ]:
data.visualize(data.images[:15], name='Image data', smooth=True)